# Antibody levels households

Check whether antibody levels in a household are more similar than one would expect statistically.

## Data preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from copy import deepcopy
import multiprocessing as mp
from jabbar import jabbar
from pyprojroot import here
import os

import sys
base_path = str(here("", project_files=[".here"]))
perm_path = os.path.join(base_path, "PermutationStudies")
if perm_path not in sys.path:
    sys.path.insert(0, perm_path)
from src.functions import *

%matplotlib inline

random.seed(0)
np.random.seed(0)

# control variable of interest
var = 'hh_id'
# measurements to study
data_key = 'R2_Result'
# number of permutations
n_perm = %%n_perm%%

# identifier
id_ = f"hh_{data_key}_{n_perm}"

In [ ]:
data = read_blab(base_path)
print(data.shape, "initially")

# remove duplicate columns
data = data.drop_duplicates(subset=['ind_id'], keep='first')
print(data.shape, "after remove duplicates")

# remove nans   
data = data[data[data_key].notnull()]
print(data.shape, "after remove nans")

# translate results
data[data_key] = (data[data_key] == "Positive").astype(float)

# data plot
fig, ax = plt.subplots(figsize=(3, 3))
ax.hist(data[data_key], color='C0', bins=100)
ax.set_xlabel(data_key)
ax.set_ylabel("Frequency")
fig.tight_layout()

Convert to numpy for efficiency

In [ ]:
%%time
data_arr = get_data_arr(data, data_key)
var_ids, var_ids_uq, var_id_matrix, var_sizes = get_var_id_stuff(data, var)
data_matrix = create_data_matrix(data_arr, var_id_matrix)
# control
print(data_matrix.shape)

In [ ]:
plt.hist(var_sizes, bins=np.arange(var_sizes.max()+1)+1, align='left')
plt.title("Household size distribution")
plt.xlabel("Household size [participants]")
plt.ylabel("Number")

## Define statistics

Since we perform a time-intensive permutation test, we need to make the computations efficient. To check validity and efficiency, below we provide implementations in pandas, numpy, and fully vectorized numpy.

Pandas implementation:

In [ ]:
%%time
real_variance = statistic_var_pd(data, data_key, var, var_ids_uq)
print(real_variance)

Numpy implementation with iteration:

In [ ]:
%%time
real_variance = statistic_var_np_iter(data_arr, var_ids, var_ids_uq)
print(real_variance)

Completely vectorized numpy implementation:

In [ ]:
%%time
data_matrix_dict = create_data_matrix(data_arr, var_id_matrix)

real_variance = statistic_var(data_matrix, var_sizes, var_id_matrix)
print(real_variance)

This >30 fold speed-up should be enough for the moment.

In [ ]:
# check whether the methods agree on permutated data

data_arr_perm = data_arr[np.random.permutation(len(data_arr))]

perm_variance = statistic_var_np_iter(data_arr_perm, var_ids, var_ids_uq)
print(perm_variance)

data_matrix_perm = create_data_matrix(data_arr_perm, var_id_matrix)
perm_variance = statistic_var(data_matrix_perm, var_sizes, var_id_matrix)
print(perm_variance)

### Permutation test

In [ ]:
%%time

# set random seed for reproducibility
np.random.seed(0)

# define permutations
data_arr_perms = get_data_arr_perms(data_arr, n_perm)

# for results
variances = []

# loop over all permutations
for data_arr_perm in jabbar(data_arr_perms, symbols='🦄'):
    # get permutation
    data_matrix = create_data_matrix(data_arr_perm, var_id_matrix)
    # compute mean variance
    variances.append(statistic_var(data_matrix, var_sizes, var_id_matrix))

# to numpy arrays
variances = np.array(variances)

In [ ]:
#save data
save_data(id_, variances=variances,
          real_variance=real_variance,
          perm_path=perm_path)

## Analysis

In [ ]:
# load data
variances, real_variance = load_data(
    id_=id_, obj_keys=['variances', 'real_variance'],
    perm_path=perm_path)

In [ ]:
# plot for variances
plot_kde(samples=variances, obj_key='variances', real_sample=real_variance,
         data_key=data_key, id_=id_, suptitle="Average variance over households",
         perm_path=perm_path)
plot_hist(samples=variances, obj_key='variances', real_sample=real_variance,
          data_key=data_key, id_=id_, suptitle="Average variance over households",
          perm_path=perm_path)

In [ ]:
print("Percentiles:")
print("Variance", data_key, sum(variances <= real_variance) / len(variances))